<a href="https://colab.research.google.com/github/KumaranShivam5/improved-succotash/blob/master/assignment2/ML_assign2_q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SPAM/HAM classification using Naive Bays classifier**

In [0]:
import numpy as np
import pandas as pd

In [0]:
## reading the data from file ##
data=pd.read_csv('spam.csv', sep="," , encoding='latin-1')
#print(data)
data=data.drop([ 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1)
data=data.rename(columns={'v1':'ctg','v2':'content'})
data.describe()
data.groupby('ctg').describe()

content                                                               
       count unique                                                top freq
ctg                                                                        
ham     4825   4516                             Sorry, I'll call later   30
spam     747    653  Please call our customer service representativ...    4

Train  test Split routine


In [0]:
#tokenizing words
#converting to lowercase
#removing stopwords

import string as st
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
def process(message):
    m_void=''
    for char in message:
        if char not in st.punctuation:
            m_void+=(char)
    m_void_token=[word.lower() for word in m_void.split()]
    m_void_token=[word for word in m_void_token if word not in stopwords.words('english')]
    return(m_void_token)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#randomixing data before splitting
data=data.sample(frac=1).reset_index(drop="true")
#print(data)
split=0.8
split_no=int(len(data)*split)
train_msg=data.iloc[0:split_no]['content']
train_labels=data.iloc[0:split_no]['ctg']
test_msg=data.iloc[split_no+1:]['content']
test_labels=data.iloc[split_no+1:]['ctg']
#print(train_data.iloc[100].content)


**Text-Processing**

In [0]:
##VECTORIZE
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer(analyzer=process)

#using only the training data for generating
#bag of words
train_data=count_vect.fit_transform(train_msg)
test_data=count_vect.transform(test_msg)

**Designing my own NB classifier**

In [0]:
def nb_fit(train_data,train_labels):
    ##normalixing frequencies
    train_data_arr=train_data.toarray()
    train_data_arr_norm=np.where(train_data_arr > 0, 1, 0)
    #vector of occurence of each words in spam
    spam_train_vector_norm=train_data_arr_norm[train_labels=='spam'].sum(axis=0)
    #vector of occurence of each words in ham
    ham_train_vector_norm=train_data_arr_norm[train_labels=='ham'].sum(axis=0)


    #calculating prior probabilities
    no_of_spam=len(train_labels[train_labels=='spam'])
    no_of_ham=len(train_labels[train_labels=='ham'])
    no_of_features=len(train_data_arr_norm[0])
    p_spam=np.log(np.divide(no_of_spam,(no_of_spam+no_of_ham)))
    p_ham=np.log(np.divide(no_of_ham,(no_of_spam+no_of_ham)))
    
    #calculating log likelihood of each words
    log_lik_spam=np.log(np.divide(spam_train_vector_norm +1, no_of_spam+no_of_features))
    log_lik_ham=np.log(np.divide(ham_train_vector_norm +1 , no_of_ham+no_of_features))
    #returns -- log-prior(ham), log-prior(spam)
    #-----log-likelihood(each words| ham),
    #-----log-likelihood(each_eords|spam)
    return(p_ham,p_spam,log_lik_ham,log_lik_spam)

def predict(test_data,model):
    test_data_arr=test_data.toarray()
    test_result=[]
    for i in range(len(test_data_arr)):
        #dot product is interesting here
        #by this we are actually adding the
        #log-likelihood of only the words 
        #that are present in the test message
        #as test message is also encoded in the
        # dictionary format as per the train data
        p_ham=model[0]+np.dot(test_data_arr[i],model[2])
        p_spam=model[1]+np.dot(test_data_arr[i],model[3])
        if(p_ham>p_spam):
            test_result.append('ham')
        else:
            test_result.append('spam')
    return(test_result)

In [0]:
model=nb_fit(train_data,train_labels)

test_prediction=predict(test_data,model)
print(np.asarray(test_prediction).shape)
print(test_labels.values.shape)
acc_mat=test_labels.values==np.asarray(test_prediction)
accuracy=len(acc_mat[acc_mat==True])/len(acc_mat)
print(accuracy*100)

(1114,)
(1114,)
97.48653500897666
